In [1]:
# import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

True

# 定义模型

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'glm-4-flash',
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
)


In [13]:
import os
from langchain_openai import ChatOpenAI
api_key = os.getenv("OPENAI_API_KEY")
# print(api_key)
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4-flash",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)



# 定义工具

## 搜索工具

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

## 数据库操作工具

In [4]:
import pandas as pd
from pandasql import sqldf

def simulate_database_operation(sql):
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result
    

In [5]:
print(simulate_database_operation('SELECT * FROM my_table WHERE age > 50'))


              name  age sex
0      Henry Myers   60   F
1      Kelsey Lutz   54   M
2   Jonathan Young   76   F
3  Kimberly Macias   69   M


In [6]:
print(simulate_database_operation('SELECT * FROM my_table WHERE id > 50'))


PandaSQLException: (sqlite3.OperationalError) no such column: id
[SQL: SELECT * FROM my_table WHERE id > 50]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [7]:
from langchain.tools import tool


@tool
def simulate_database_operation(sql: str):
	'''根据sql语句操作数据库'''
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result



## 模型调用工具

In [8]:
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
model_with_tools = model.bind_tools(tools)



In [9]:
from pprint import pprint as pp

response = model_with_tools.invoke('印度的首都是哪里？')
pp(dict(response))


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}

In [14]:
fetch_response = model_with_tools.invoke('现在是北京时间几点？查询之后告诉我')
pp(dict(fetch_response))


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}

In [15]:
db_response = model_with_tools.invoke('帮我往数据库的my_table表中插入一条数据，name是张三，age是18，sex是male')
pp(dict(db_response))


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}